In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm
import torch
import os

# ✅ 모델 및 토크나이저 설정
model_name = "paust/pko-t5-base"  # 또는 'digit82/kot5-paraphrase', 'KETI-AIR/ke-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 변환할 스타일 목록
styles = ['formal', 'informal', 'gentle', 'king', 'enfp']

# ✅ 중립적 입력 문장 리스트
source_sentences = [
    "오늘은 날씨가 정말 좋다.",
    "회의가 너무 길어서 피곤하다.",
    "지금 집에 가는 중이야.",
    "배가 너무 고프다.",
    "이번 주말엔 푹 쉬고 싶어.",
    "과제가 너무 많아서 스트레스 받아.",
    "친구랑 오랜만에 만나서 기분이 좋았다.",
    "영화를 보고 나니까 눈물이 났어.",
    "오늘은 정말 정신없는 하루였다.",
    "지친 몸을 이끌고 집에 돌아오니 마음이 편안해졌다."
]

# ✅ 스타일 문장 생성 함수
def generate_stylized_sentence(style, source, max_length=64):
    prompt = f"Translate to {style} style: {source}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            early_stopping=True
        )
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # ❗후처리: 프롬프트 흔적 제거
    if "Translate" in result or "style" in result:
        result = result.replace("Translate", "").replace("style", "").strip(": ").strip()
    return result


# ✅ 전체 생성 및 저장
results = []
for source in tqdm(source_sentences, desc="스타일 문장 생성 중"):
    for style in styles:
        target = generate_stylized_sentence(style, source)
        results.append({
            "style": style,
            "source": source,
            "target": target
        })

# ✅ CSV로 저장 (한글 깨지지 않도록)
output_path = "generated_style_dataset.csv"
df = pd.DataFrame(results)
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 스타일 문장 CSV 저장 완료: {os.path.abspath(output_path)}")


스타일 문장 생성 중: 100%|██████████| 10/10 [00:25<00:00,  2.58s/it]

✅ 스타일 문장 CSV 저장 완료: c:\Users\AI-LJH\Desktop\캡스톤\generated_style_dataset.csv
